# Group Theory Homework: A Study of Point Group Symmetries in Molecules

A notebook that lays out plainly all the code used to complete the associated homework assg. Written by Theo N. Dionne.

In [1]:
# imports
import numpy as np

## Calculating the decomposition of the mechanical reps

In [2]:
# functions
def get_irrep_multiplicity(representation_characters, character_table, elements_per_class):
    """A function that gets the multiplicity of each irrep 
       for real representations using the magic formula.
    """
    # multiply each character by the number of elements in its class
    representation_characters_adjusted = representation_characters*elements_per_class

    # perform a matrix multiplication between the character table and divide by group size
    multiplicity_vector = character_table @ representation_characters_adjusted / np.sum(elements_per_class)

    return multiplicity_vector

### AB4 (C4v)

In [3]:
# define the character table of the PG
character_table_C4v = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1,-1,-1],
    [1, 1,-1, 1,-1],
    [1, 1,-1,-1, 1],
    [2,-2, 0, 0, 0]
])  

# define the number of elements per class
elements_per_class_C4v = np.array([1,1,2,2,2])

# the characters of the mechanical representation
M_characters_C4v = np.array([15,-1,1,1,3])

# Getting and printing the mutliplicities
M_multiplicities_C4v = get_irrep_multiplicity(M_characters_C4v, character_table_C4v, elements_per_class_C4v)
print(M_multiplicities_C4v) # in the same order as the table in the pdf

[3. 1. 1. 2. 4.]


### UF6 (Oh)

In [4]:
# define the character table of the PG
character_table_Oh = np.array([
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 1, 1, 1, 1,-1,-1,-1,-1,-1],
[1, 1,-1,-1, 1, 1, 1,-1,-1, 1],
[1, 1,-1,-1, 1,-1,-1, 1, 1,-1],
[2, 2, 0, 0,-1, 2, 2, 0, 0,-1],
[2, 2, 0, 0,-1,-2,-2, 0, 0, 1],
[3,-1, 1,-1, 0, 3,-1, 1,-1, 0],
[3,-1, 1,-1, 0,-3, 1,-1, 1, 0],
[3,-1,-1, 1, 0, 3,-1,-1, 1, 0],
[3,-1,-1, 1, 0,-3, 1, 1,-1, 0]
])  

# define the number of elements per class
elements_per_class_Oh = np.array([1, 3, 6, 6, 8, 1, 3, 6, 6, 8])

# the characters of the mechanical representation
M_characters_Oh = np.array([21,-3, 3,-1, 0,-3, 5,-1, 3, 0])

# Getting and printing the mutliplicities
M_multiplicities_Oh = get_irrep_multiplicity(M_characters_Oh, character_table_Oh, elements_per_class_Oh)
print(M_multiplicities_Oh) # in the same order as the table in the pdf

[1. 0. 0. 0. 1. 0. 1. 3. 1. 1.]


## Getting symmetry adapted modes

In [114]:
# functions

def _get_swap_matrices(atom_coordinates, vector_rep):
    """A function that gets the matrices (swap matrices) that define 
    how the atoms are swapped under the action of the group. """

    # generate empty swap matrix array
    number_of_operations = np.shape(vector_rep)[0] # number of group elements considered
    number_of_atoms = np.shape(atom_coordinates)[1] # number of atoms considered
    swap_matrices = np.zeros((number_of_operations, number_of_atoms, number_of_atoms))

    # applies every transformation to the atom coordinates
    transformed_coordinates = vector_rep @ atom_coordinates

    for i in range(number_of_operations):
        for j in range(number_of_atoms):
            for k in range(number_of_atoms):
                norm_transformed_minus_reference = np.linalg.norm(transformed_coordinates[i,:,k] - atom_coordinates[:,j])
                if norm_transformed_minus_reference == 0:
                    # set swap matrix
                    swap_matrices[i,k,j] = 1
                    break
        
    return swap_matrices, number_of_operations, number_of_atoms


def get_mechanical_representation(atom_coordinates, vector_rep):
    """A function that calculates the mechanical representation based off the fact 
    that M = S \\otimes V where S is the swap matrix and V is the vector representation"""

    # obtain swap matrices
    swap_matrices, number_of_operations, number_of_atoms = _get_swap_matrices(atom_coordinates, vector_rep)
    
    # initialize mechanical repr matrices
    m_rep_size = 3*number_of_atoms
    mechanical_rep = np.zeros((number_of_operations, m_rep_size, m_rep_size))
    
    # generate the matrices
    for i in range(number_of_operations):
        mechanical_rep[i,:,:] = np.kron(swap_matrices[i,:,:], vector_rep[i,:,:])

    return mechanical_rep


In [115]:
atom_spatial_coordinates_C4v = np.array([
    [ 0, 1, 0,-1, 0],
    [ 0, 0, 1, 0,-1],
    [ 1, 0, 0, 0, 0]
])

In [116]:
# Writting down the matrices from genpos
vector_rep_C4v = np.array([
    [
        [ 1, 0, 0],
        [ 0, 1, 0],
        [ 0, 0, 1]
    ],
    [
        [-1, 0, 0],
        [ 0,-1, 0],
        [ 0, 0, 1]
    ],
    [
        [ 0,-1, 0],
        [ 1, 0, 0],
        [ 0, 0, 1]
    ],
    [
        [ 0, 1, 0],
        [-1, 0, 0],
        [ 0, 0, 1]
    ],
    [
        [ 1, 0, 0],
        [ 0,-1, 0],
        [ 0, 0, 1]
    ],
    [
        [-1, 0, 0],
        [ 0, 1, 0],
        [ 0, 0, 1]
    ],
    [
        [ 0,-1, 0],
        [-1, 0, 0],
        [ 0, 0, 1]
    ],
    [
        [ 0, 1, 0],
        [ 1, 0, 0],
        [ 0, 0, 1]
    ]
])

# print matrices to check em out
for i in range(np.shape(vector_rep_C4v)[0]):
    print(vector_rep_C4v[i,:,:])

print(vector_rep_C4v[2,:,:] @ atom_spatial_coordinates_C4v)

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[-1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  1]]
[[ 0  1  0]
 [-1  0  0]
 [ 0  0  1]]
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
[[-1  0  0]
 [ 0  1  0]
 [ 0  0  1]]
[[ 0 -1  0]
 [-1  0  0]
 [ 0  0  1]]
[[0 1 0]
 [1 0 0]
 [0 0 1]]
[[ 0  0 -1  0  1]
 [ 0  1  0 -1  0]
 [ 1  0  0  0  0]]


In [123]:
get_mechanical_representation(atom_spatial_coordinates_C4v, vector_rep_C4v)

array([[[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]],

       [[-1.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0., -1.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ...,
        [-0.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0., -0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0., -1.,  0., ...,  0., -0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ...,
        [ 0., -0.,  0., ...,  0., -0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ...,

       [[-1.,  0.,  0., ..., -0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.